# Gaussian Mixture Models

### Motivação

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=400, centers=4,
                       cluster_std=0.60, random_state=0)
X = X[:, ::-1] # flip axes for better plotting

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(4, random_state=0)
labels = kmeans.fit(X).predict(X)
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

In [ ]:
from scipy.spatial.distance import cdist

def plot_kmeans(kmeans, X, n_clusters=4, rseed=0, ax=None):
    labels = kmeans.fit_predict(X)

    # plot the input data
    plt.figure(figsize=(8, 8), dpi=80)
    ax = ax or plt.gca()
    ax.axis('equal')
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)

    # plot the representation of the KMeans model
    centers = kmeans.cluster_centers_
    radii = [cdist(X[labels == i], [center]).max()
             for i, center in enumerate(centers)]
    for c, r in zip(centers, radii):
        ax.add_patch(plt.Circle(c, r, fc='#CCCCCC', lw=3, alpha=0.5, zorder=1))

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X)

In [ ]:
rng = np.random.RandomState(13)
X_stretched = np.dot(X, rng.randn(2, 2))

kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X_stretched)

## Introdução

In [ ]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

In [ ]:
#vetor de pesos (fi)
gm.weights_

In [ ]:
gm.means_

In [ ]:
gm.covariances_

Voltando aos dados do exemplo de motivação

In [ ]:
X, y_true = make_blobs(n_samples=400, centers=4,
                       cluster_std=0.60, random_state=0)
X = X[:, ::-1] # flip axes for better plotting

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=4).fit(X)
labels = gmm.predict(X)
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

In [ ]:
probs = gmm.predict_proba(X)
print(probs[:5].round(3))

In [ ]:
size = 50 * probs.max(1) ** 2  # square emphasizes differences
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=size);

In [ ]:
gmm.converged_

In [ ]:
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [ ]:
gmm = GaussianMixture(n_components=4, random_state=42)
plot_gmm(gmm, X)

In [ ]:
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=42)
plot_gmm(gmm, X_stretched)

### Fazendo predição de novos dados

In [ ]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

In [ ]:
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

In [ ]:
gm.predict(X)

In [ ]:
gm.predict_proba(X)

In [ ]:
X_new, y_new = gm.sample(6)
X_new

In [ ]:
y_new

### Density Estimation

In [ ]:
from sklearn.datasets import make_moons
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
plt.scatter(Xmoon[:, 0], Xmoon[:, 1]);

In [ ]:
gmm_ds = GaussianMixture(n_components=2, covariance_type='full', random_state=0)
plot_gmm(gmm_ds, Xmoon)

In [ ]:
gmm16 = GaussianMixture(n_components=16, covariance_type='full', random_state=0)
plot_gmm(gmm16, Xmoon, label=False)

In [ ]:
Xnew = gmm16.sample(400)
plt.scatter(Xnew[0][:,0], Xnew[0][:,1]);

### Tipo de Covariância

In [ ]:
#função auxiliar definida na aula do kmeans para visualizar os centróides
def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

In [ ]:
# função auxiliar para plotar as fronteiras de decisão de uma GMM
from matplotlib.colors import LogNorm

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='r', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
#função auxiliar para comparar, 2 a 2, os diferentes tipos de covariância
def compare_gaussian_mixtures(gm1, gm2, X):
    plt.figure(figsize=(9, 4))

    plt.subplot(121)
    plot_gaussian_mixture(gm1, X)
    plt.title('covariance_type="{}"'.format(gm1.covariance_type), fontsize=14)

    plt.subplot(122)
    plot_gaussian_mixture(gm2, X, show_ylabels=False)
    plt.title('covariance_type="{}"'.format(gm2.covariance_type), fontsize=14)

In [ ]:
# instanciando os 4 tipos de covariância
gm_full = GaussianMixture(n_components=3, n_init=10, covariance_type="full", random_state=42)
gm_tied = GaussianMixture(n_components=3, n_init=10, covariance_type="tied", random_state=42)
gm_spherical = GaussianMixture(n_components=3, n_init=10, covariance_type="spherical", random_state=42)
gm_diag = GaussianMixture(n_components=3, n_init=10, covariance_type="diag", random_state=42)
gm_full.fit(X)
gm_tied.fit(X)
gm_spherical.fit(X)
gm_diag.fit(X)

In [ ]:
# comparando tied e spherical
compare_gaussian_mixtures(gm_tied, gm_spherical, X)
plt.show()

In [ ]:
compare_gaussian_mixtures(gm_full, gm_diag, X)
plt.tight_layout()
plt.show()

## Detecção de Anomalia usando GMM

In [ ]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

In [ ]:
plt.figure(figsize=(8, 4))

plot_gaussian_mixture(gm, X)
plt.scatter(anomalies[:, 0], anomalies[:, 1], color='y', marker='*')
plt.ylim(top=5.1)

plt.show()

## Selecionando o número de Cluster

In [ ]:
gm.bic(X)

In [ ]:
gm.aic(X)

Vamos treinar vários GMM com vários valores de k e mensurar BIC e AIC

In [ ]:
gms_per_k = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(X)
             for k in range(1, 11)]

In [ ]:
bics = [model.bic(X) for model in gms_per_k]
aics = [model.aic(X) for model in gms_per_k]

In [ ]:
#agora plotamos os valores para verificar o melhor valor de k
plt.figure(figsize=(8, 3))
plt.plot(range(1, 11), bics, "bo-", label="BIC")
plt.plot(range(1, 11), aics, "go--", label="AIC")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Information Criterion", fontsize=14)
plt.axis([1, 9.5, np.min(aics) - 50, np.max(aics) + 50])
plt.annotate('Minimum',
             xy=(3, bics[2]),
             xytext=(0.35, 0.6),
             textcoords='figure fraction',
             fontsize=14,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.legend()
plt.show()

Por último, podemos procurar pela melhor combinação de valores tanto para o número de clusters quanto o tipo de covariância

In [ ]:
min_bic = np.infty

for k in range(1, 11):
    for covariance_type in ("full", "tied", "spherical", "diag"):
        bic = GaussianMixture(n_components=k, n_init=10,
                              covariance_type=covariance_type,
                              random_state=42).fit(X).bic(X)
        if bic < min_bic:
            min_bic = bic
            best_k = k
            best_covariance_type = covariance_type

In [ ]:
best_k

In [ ]:
best_covariance_type

# Topic Modelling usando Latent Dirichlet Allocation (LDA)

Algumas considerações antes de estudarmos LDA:

* Como a disciplina não é de NLP (Processamento de Linguagem Natural), alguns métodos que utilizei aqui não serão explicados em detalhes, mas tentarei ser o mais didádito possível para torná-los compreensíveis

* Gensim é uma biblioteca voltada para NLP que possui vários métodos de tratamento de textos e algoritmos para transformar textos

* NLTK é um toolkit que contém vários métodos de tratamento de textos e dicionários incorporados

In [ ]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
import numpy as np
np.random.seed(42)
from gensim import corpora, models
from pprint import pprint

In [ ]:
data = pd.read_csv('bases/abcnews-date-text.csv', error_bad_lines=False, nrows=500000)
print(data.head())
documents = data[['headline_text']]
documents['index'] = documents.index
print()
documents.head()

In [ ]:
len(documents)

### Pré-processamento dos dados

É uma etapa crucial na análise de textos, em que tratamos eles de maneira correta para usa-los como input para algoritmos de machine learning. Aqui, vamos executar os seguintes passos:

1. Tokenização: divide o texto em sentenças e as sentenças em palavras; remove toda pontuação e deixa todas as palavras em minúscula

2. Elimina palavra que possuem menos de 3 caracteres

3. Elimina as stopwords

4. Lematização e Stemização

A lematização é o processo de tornar uma palavra em sua raiz, conforme exemplo:

In [ ]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))
# pos = part of speech -> tageamento de palavras conforme sua classe gramatical

Stemização é o processo de eliminar os radicais de uma palavra, conforme exemplos:

In [ ]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

Vamos escrever duas funções:

1. Aplicação de lemmatização e stemização nas palavras

2. pré-processamento do texto conforme indicado acima

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

Vamos pegar um exemplo e comparar sua forma original com sua forma tratada:

In [ ]:
doc_sample = documents[documents['index'] == 620].values[0][0]

print('Documento original: ')
print(documents[documents['index'] == 620])
print()
print('Palavras do Documento original')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n Documento processado: ')
print(preprocess(doc_sample))

Vamos aplicar o pré-processamento a todo o texto agora:

In [ ]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:5]

### Bag-of-Words

* A partir de processed_docs, vamos criar um dicionário que contém o número de vezes que uma palavra apareceu no conjunto de treino

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

Agora, vamos aplicar um filtros nos tokens que aparecem em:

1. menos que 15 números;

2. mais que 0.5 documentos (fração do corpus total (total de documentos))

3. depois dos dois primeiros passos, mantém apenas os primeiros 100000 tokens mais frequentes

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

### doc2bow

Para cada documento, criamos um dicionário que relata palavras e quantas vezes elas apareceram:

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[620]

Vamos olhar como fica nosso exemplo pre-processado:

In [ ]:
bow_doc_620 = bow_corpus[620]

for i in range(len(bow_doc_620)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_620[i][0], 
                                                     dictionary[bow_doc_620[i][0]], 
                                                     bow_doc_620[i][1]))

## TF-IDF

* TF (Term-Frequency): quanto mais uma palavra aparece num documento, mais relevante ela é para descrever esse documento

* IDF (Inverse-Document Frequency): palavras que aparecem raras vezes num documento também podem ser bons descritores desse documento

A ideia do TF-IDF é ponderar a relevância das palavras num documento

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
for doc in corpus_tfidf:
    pprint(doc)
    break

## Executando LDA usando Bag-of-Words

Após realizar o tratamento dos textos, vamos agora executar o LDA e analisar seus resultados. 

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

Para cada tópilo, vamos explorar a ocorrência de palavras neste tópico e seu peso relativo:

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

### Executando LDA usando TF-IDF
Vamos executar o LDA usando a outra abordagem agora

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

## Avaliação dos Modelos
Vamos agora avaliar os modelos

### Usando Bag-of-Words

In [ ]:
for index, score in sorted(lda_model[bow_corpus[620]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

### Usando TF-IDF

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[620]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

## Testando o modelo em documentos não vistos

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))